In [1]:
import csv
import json
import math
import pprint
import pytz

from datetime import datetime, timedelta

In [2]:
def timezone_code_to_olson(timezone_code):
    """
    Converts a three-letter timezone code to an Olson timezone name for US timezones.
    Returns None if the timezone code is not recognized.
    """
    if timezone_code == 'EST':
        return 'America/New_York'
    elif timezone_code == 'EDT':
        return 'America/New_York'
    elif timezone_code == 'CST':
        return 'America/Chicago'
    elif timezone_code == 'CDT':
        return 'America/Chicago'
    elif timezone_code == 'MST':
        return 'America/Denver'
    elif timezone_code == 'MDT':
        return 'America/Denver'
    elif timezone_code == 'PST':
        return 'America/Los_Angeles'
    elif timezone_code == 'PDT':
        return 'America/Los_Angeles'
    else:
        return None
    
    
def string_to_utc_dt(date_str, date_format):
    """
    Converts a date string with its format into a UTC datetime
    """
    tz_code = date_str[-4:-1]
    tz_olson = timezone_code_to_olson(tz_code)
    local_tz = pytz.timezone(tz_olson)
    
    raw_dt = datetime.strptime(date_str, date_format)
    local_dt = local_tz.localize(raw_dt)
    utc_dt = local_dt.astimezone(pytz.utc)
    
    return utc_dt

    
    

In [3]:
def clean_chargepoint_data(filename):
    """
    Import chargepoint csv download
    """
    with open("inputs/" + filename) as file:
        reader = csv.DictReader(file)
        charge_data = [x for x in reader]    

    input_date_format = '%Y-%m-%d %H:%M:%S (%Z)'
    
    for row in charge_data:
        
        # convert dates to datetimes in UTC
        row['start_dt'] = string_to_utc_dt(row["Session Start Date"], input_date_format)
        row['end_dt'] = string_to_utc_dt(row["Session End Date"], input_date_format)

        # convert string kwh to float
        row['kwh'] = float(row["Session Energy (kWh)"])
    
    return charge_data
        
charging_sessions = clean_chargepoint_data("Usage-history-20230424.csv")

In [6]:
# rate of charge when vehicle is plugged in
# Note: there's some fluctuation and a slight ramp so this isn't perfect
# and will vary from vehicle to vehicle
charge_rate_kw = 9.2

balancing_authority = 'ERCO'


def backtest_sessions(sessions, rate, ba):
    
    # get emissions for the balancing authority
    with open(f"data/emissions/clean/avg_hourly_emissions/{ba}.csv") as file:
        reader = csv.DictReader(file)
        grid_hours = [x for x in reader]
        
    for hour in grid_hours:
        hour['ts'] = datetime.fromisoformat(hour['ts'])
        hour['avg'] = float(hour['avg'])
        
    last_ts = grid_hours[-1]['ts']

    # for each charging session
    results = []
    
    for session in sessions:
        
        # skip sessions where we don't have grid emissions data yet
        if session['end_dt'] > last_ts:
            continue

        # figure out how many hours you need (total kwh / charging_rate_kw)
        hours_needed = math.ceil(session['kwh'] / rate)
        
        # get all hours during that session
        if hours_needed == 1:
            session_grid_factors = [x['avg'] for x in grid_hours if x['ts'] >= session['start_dt'] - timedelta(hours=1) and x['ts'] <= session['end_dt']]
        else:
            session_grid_factors = [x['avg'] for x in grid_hours if x['ts'] >= session['start_dt'] and x['ts'] <= session['end_dt']]
        
        # get emissions for first X hours
        actual_window = session_grid_factors[:hours_needed]
        actual_factor = sum(actual_window) / hours_needed
        
        # find the lowest adjacent hours
        optimal_window = actual_window
        optimal_factor = actual_factor
        optimal_start = 0
        
        if len(session_grid_factors) > hours_needed:
            for i in range(1, len(session_grid_factors) - (hours_needed - 1)):
                window = session_grid_factors[i:i+hours_needed]
                factor = sum(window) / hours_needed
                if factor < optimal_factor:
                    optimal_window = window
                    optimal_factor = factor
                    optimal_start = i
                    
        
        session['emissions_actual'] = actual_factor * session['kwh']
        session['emissions_optimal'] = optimal_factor * session['kwh']
        session['emissions_pct_change'] = round(((session['emissions_optimal'] / session['emissions_actual']) - 1) * 100, 1)
        
        session['hours_needed'] = hours_needed
        session['session_grid_factors'] = session_grid_factors
        session['optimal_start_hour'] = optimal_start
        
        results.append(session)
        
    print('done.')
    return results

results = backtest_sessions(charging_sessions, charge_rate_kw, balancing_authority)



done.


In [18]:
def viz_results(sessions):
    
    # ours is actually closer to 2.6, but this should be more typical of other BEVs
    total_kwh = sum([x['kwh'] for x in sessions])
    mpkw = 3.0
    total_miles = total_kwh * mpkw
    
    total_emissions_actual = sum([x['emissions_actual'] for x in sessions])
    total_emissions_optimal = sum([x['emissions_optimal'] for x in sessions])
    total_pct_savings = round(((total_emissions_optimal / total_emissions_actual) - 1) * 100, 1)
    
    print("SUMMARY\n-------")
    print(f"Total KWHs:  {round(total_kwh, 1)}")
    print(f"Total miles: {round(total_miles, 1)}")
    print()
    print(f"Total emissions reduction potential: {total_pct_savings}%")
    print("")
    
    avg_cost_per_kwh = 0.11
    avg_miles_per_year = 13500
    illustrative_cost = (avg_miles_per_year / mpkw) * avg_cost_per_kwh
    illustrative_cost_savings = illustrative_cost * (total_pct_savings / 100)
    
    print(f"Illustrative annual cost:         ${round(illustrative_cost, 2)}")
    print(f"Illustrative annual cost savings: ${round(illustrative_cost_savings, 2)}")
    print('\n')
    
    print("CHARGING SESSION DETAIL\n--------------------")
    for session in sessions:
        
        # output
        print(f"kw charge:         {session['kwh']}")
        print(f"hours needed:      {session['hours_needed']}")
        print(f"hours available:   {len(session['session_grid_factors'])}")
        print()
        print(f"actual emissions:  {round(session['emissions_actual'], 1)}")
        print(f"optimal emissions: {round(session['emissions_optimal'], 1)}")
        print(f"reduction:         {session['emissions_pct_change']}%")
        print()
        
        viz_actual = ""
        viz_optimal = ""
        for i in range(0, len(session['session_grid_factors'])):
            if i < session['hours_needed']:
                viz_actual += '  *  '
            else:
                viz_actual += "  -  "
            
            if i >= session['optimal_start_hour'] and i < session['optimal_start_hour'] + session['hours_needed']:
                viz_optimal += '  *  '
            else:
                viz_optimal += '  -  '

        print("lb CO2e/kwh:  " + ' '.join(map(str, [round(x, 2) for x in session['session_grid_factors']])))
        print("actual:       " + viz_actual)
        print("optimal:      " + viz_optimal)
        

        print()
        print('-------') 

viz_results(results)

SUMMARY
-------
Total KWHs:  1426.3
Total miles: 4278.9

Total emissions reduction potential: -7.5%

Illustrative annual cost:         $495.0
Illustrative annual cost savings: $-37.12


CHARGING SESSION DETAIL
--------------------
kw charge:         37.903
hours needed:      5
hours available:   12

actual emissions:  23.9
optimal emissions: 23.0
reduction:         -3.7%

lb CO2e/kwh:  0.68 0.65 0.64 0.61 0.58 0.59 0.62 0.68 0.76 0.8 0.89 0.93
actual:         *    *    *    *    *    -    -    -    -    -    -    -  
optimal:        -    -    *    *    *    *    *    -    -    -    -    -  

-------
kw charge:         35.904
hours needed:      4
hours available:   12

actual emissions:  22.8
optimal emissions: 19.4
reduction:         -14.9%

lb CO2e/kwh:  0.71 0.66 0.61 0.56 0.55 0.53 0.53 0.56 0.64 0.72 0.78 0.81
actual:         *    *    *    *    -    -    -    -    -    -    -    -  
optimal:        -    -    -    -    *    *    *    *    -    -    -    -  

-------
kw charge:     